In [15]:
from pynq import Overlay
import pynq.lib.dma
# Load the overlay
overlay = Overlay('/home/xilinx/pynq/overlays/FSRCNN/FSRCNN.bit')
#overlay?
# Load the DMA
dma = overlay.DMA 
# Load FSRCNN IP
FSRCNN_ip = overlay.FSRCNN_0
#test_ip?

In [16]:
import cv2
import numpy as np
in_img = cv2.imread('input_image_64x64_butterfly.jpeg')
height, width, nr_of_channels = in_img.shape
print('Input image size: ',in_img.shape)

Input image size:  (64, 64, 3)


In [17]:
#write the constants to the respective addresses
scale_factor = 2
FSRCNN_ip.write(0x10,height)                            
FSRCNN_ip.write(0x18,width)  
FSRCNN_ip.write(0x20,scale_factor)                                                  
#verif_input_width = FSRCNN_ip.read(0x18)
verif_scale_factor = FSRCNN_ip.read(0x20)
print('Scale factor: ',verif_scale_factor)

Scale factor:  2


In [18]:
from   pynq import allocate
import time
out_height = scale_factor*height
out_width  = scale_factor*width
size_stream_in    = height*width
size_stream_out   = out_height*out_width
red_channel_in    = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
red_channel_out   = allocate(shape=(size_stream_out,1), dtype=np.uint32)
green_channel_in  = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
green_channel_out = allocate(shape=(size_stream_out,1), dtype=np.uint32)
blue_channel_in   = allocate(shape=(size_stream_in,1),  dtype=np.uint32)
blue_channel_out  = allocate(shape=(size_stream_out,1), dtype=np.uint32)
k= 0
for i in range(height):
    for j in range(width):
            red_channel_in[k,0]   = in_img[i,j,0]
            green_channel_in[k,0] = in_img[i,j,1]
            blue_channel_in[k,0]  = in_img[i,j,2]
            k = k + 1

In [19]:
import time
from contextlib import suppress
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(red_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(red_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                    #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError): 
    dma.recvchannel.wait()
red_channel_hw_exec_time = stop_time - start_time

In [20]:
overlay = Overlay('/home/xilinx/pynq/overlays/FSRCNN/FSRCNN.bit')
# Load the DMA
dma = overlay.DMA 
# Load FSRCNN IP
FSRCNN_ip = overlay.FSRCNN_0

In [21]:
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(green_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(green_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                      #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError):
    dma.recvchannel.wait()
green_channel_hw_exec_time = stop_time - start_time

In [22]:
overlay = Overlay('/home/xilinx/pynq/overlays/FSRCNN/FSRCNN.bit')
# Load the DMA
dma = overlay.DMA 
# Load FSRCNN IP
FSRCNN_ip = overlay.FSRCNN_0

In [23]:
start_time = time.time()
# Trigger the DMA transfer and wait for the result
dma.sendchannel.transfer(blue_channel_in)  #python - dma - accelerator PYNQ
dma.recvchannel.transfer(blue_channel_out) #PYNQ accelerator - dma - python
dma.sendchannel.wait()                     #makes sure that does not move on until the transfer is effectuated
stop_time = time.time()
with suppress(RuntimeError):
    dma.recvchannel.wait()
blue_channel_hw_exec_time = stop_time - start_time

In [24]:
CPU_C_exec_time = 163
CPU_Python_exec_time = 273.80
hw_exec_time = red_channel_hw_exec_time + green_channel_hw_exec_time + blue_channel_hw_exec_time
print('Hardware FSRCNN execution time: ',hw_exec_time)
print('Hardware acceleration factor (FPGA accelerator vs C/C++ CPU version): ',CPU_C_exec_time / hw_exec_time)
print('Hardware acceleration factor (FPGA accelerator vs Python CPU version): ',CPU_Python_exec_time / hw_exec_time)

Hardware FSRCNN execution time:  7.0588579177856445
Hardware acceleration factor (FPGA accelerator vs C/C++ CPU version):  23.09155417185857
Hardware acceleration factor (FPGA accelerator vs Python CPU version):  38.78814436966182


In [25]:
out_img = np.zeros((out_height, out_width, nr_of_channels))
print('Output image size: ',out_img.shape)
k = 0
for i in range(out_height):
    for j in range(out_width):
            red_channel_bin = np.binary_repr(red_channel_out[k,0], width = 32)
            out_img[i,j,0] = int(str(red_channel_bin[24:32]), 2)
            green_channel_bin = np.binary_repr(green_channel_out[k,0], width = 32)
            out_img[i,j,1] = int(str(green_channel_bin[24:32]), 2)
            blue_channel_bin = np.binary_repr(blue_channel_out[k,0], width = 32)
            out_img[i,j,2] = int(str(blue_channel_bin[24:32]), 2)
            k = k+1
cv2.imwrite('output_image_128x128_butterfly.jpeg', out_img)
#cv2.imshow('output_image',out_img)
#cv2.waitKey(0)

Output image size:  (128, 128, 3)


True